# Diplomado Internacional: Modelado Matemático y Simulaciones
## Módulo 3: Fundamentos teóricos de la modelación con grafos

**Instructores:** Dr. Jesús F. Espinoza & Dra. Rosalía G. Hernández

Departamento de Matemáticas, Universidad de Sonora, México.

---

### Objetivo del Módulo
Establecer las bases teóricas de la teoría de grafos y el análisis topológico de datos (TDA) para su aplicación en modelado y simulación.

### Estructura
*   Sesión 1: Fundamentos de teoría de grafos.
*   **Sesión 2: Análisis topológico de datos (TDA).** (Contenido de esta libreta)

## Instrucciones para usar esta libreta en Google Colab

Bienvenidos a la parte práctica del módulo de Análisis Topológico de Datos. Esta libreta está diseñada para complementar los conceptos teóricos vistos en la presentación.

**Importante: No se requiere experiencia previa en programación con Python.**

La libreta contiene dos tipos de bloques (o celdas):
1.  **Bloques de Texto (Markdown):** Contienen definiciones, explicaciones y ejercicios teóricos.
2.  **Bloques de Código (Python):** Contienen ejemplos prácticos y ejercicios de programación guiados.

### ¿Cómo ejecutar el código?
*   Para ejecutar un bloque de código, haga clic sobre él y presione el botón de "Play" (▶️) que aparece a la izquierda, o use el atajo de teclado `Shift + Enter`.
*   Es fundamental ejecutar los bloques de código en orden, de arriba hacia abajo, ya que algunos bloques dependen de los anteriores.

### Ejercicios (Evaluación 📝)
*   Los ejercicios evaluables están marcados con un lápiz (📝).
*   Para los ejercicios de código, se le pedirá modificar ligeramente líneas existentes o completar código en las áreas indicadas (`<- MODIFIQUE AQUÍ`). Siga las instrucciones en los comentarios del código (texto precedido por `#`).
*   Para los ejercicios conceptuales, deberá hacer doble clic en el bloque de texto designado (marcado con `### INICIO/FIN DE SU RESPUESTA ###`) y escribir su respuesta.

## Configuración inicial y librerías

Para realizar TDA en Python, utilizaremos librerías especializadas. Las principales que usaremos son:
*   `Numpy` y `Matplotlib`: Para la manipulación de datos y visualización.
*   `Scipy` e `Itertools`: Para cálculos eficientes de distancias y combinaciones (usados en las visualizaciones).
*   `NetworkX`: Útil para visualizar grafos y complejos de cliques.
*   `Ripser`: Una librería eficiente para calcular la homología persistente.
*   `Persim`: Para visualizar los resultados (diagramas de persistencia y códigos de barras).

`Ripser` y `Persim` no vienen preinstaladas en Google Colab, por lo que debemos instalarlas primero.

**Ejecute el siguiente bloque de código** para instalar e importar estas librerías. (La instalación puede tardar un minuto).

In [ ]:
# ==================== INSTALACIÓN DE PAQUETES ====================
print("Instalando librerías necesarias...")
!pip install -q ripser persim networkx scipy
print("✓ Instalación completada.\n")

# ==================== IMPORTACIONES ====================
# Librerías estándar de Python
import warnings
from itertools import combinations

# Librerías científicas fundamentales
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
from matplotlib.collections import LineCollection, PolyCollection

# Librerías de análisis
from scipy.spatial.distance import pdist, squareform
import networkx as nx  # Para visualizar grafos y complejos de cliques

# Librerías de TDA (Topological Data Analysis)
from ripser import ripser
from persim import plot_diagrams

# Utilidades de Colab
from IPython.display import HTML

print("✓ Todas las librerías importadas correctamente.")

# Sesión 2: Análisis Topológico de Datos (TDA)

## 7. Introducción al Análisis Topológico de Datos (TDA)

### Motivación
Los datos modernos suelen ser de alta dimensión, ruidosos y complejos. El TDA proporciona herramientas basadas en la **topología algebraica** para descubrir la "forma" subyacente de los datos de manera robusta.

La topología estudia las propiedades de los objetos que se preservan bajo deformaciones continuas (estirar, doblar) sin roturas ni pegados. Por ejemplo, una dona y una taza de café son topológicamente equivalentes (ambas tienen un agujero).

### Principios del TDA
1.  **Invarianza topológica:** Busca características que dependen de la estructura de conectividad global, no de la métrica específica.
2.  **Estabilidad:** Pequeñas perturbaciones en los datos (ruido) producen solo pequeños cambios en los descriptores topológicos. Esto es crucial para aplicaciones reales.
3.  **Representaciones comprimidas:** Resume la estructura en descriptores concisos.

### Flujo de trabajo del TDA
El flujo estándar de TDA que exploraremos es el siguiente:

**Datos (Nube de puntos) $\to$ Complejo Simplicial (Filtración) $\to$ Homología Persistente $\to$ Descriptores (Diagramas/Barcodes)**

## 8. Simplejos y complejos simpliciales

Para estudiar la topología de los datos, necesitamos construir una estructura sobre ellos. Los bloques de construcción básicos en TDA son los **simplejos**.

### Simplejos: Los bloques de construcción

> **Definición.**
> Un **k-simplejo** es la generalización de un triángulo a dimensión $k$. Es la envolvente convexa de $k+1$ puntos afínmente independientes.

*   0-simplejo: Vértice (punto).
*   1-simplejo: Arista (segmento).
*   2-simplejo: Triángulo (relleno).
*   3-simplejo: Tetraedro (sólido).
*   ...

In [ ]:
# Visualización de simplejos
fig, axs = plt.subplots(1, 4, figsize=(12, 3))

# 0-simplejo
axs[0].scatter([0], [0], s=100); axs[0].set_title("0-simplejo\n(Vértice)")

# 1-simplejo
axs[1].plot([0, 1], [0, 0], 'b-'); axs[1].scatter([0, 1], [0, 0], s=100)
axs[1].set_title("1-simplejo\n(Arista)")

# 2-simplejo
puntos_triangulo = np.array([[0, 0], [1, 0], [0.5, 0.866]])
triangulo = plt.Polygon(puntos_triangulo, color='lightblue')
axs[2].add_patch(triangulo); axs[2].scatter(puntos_triangulo[:,0], puntos_triangulo[:,1], s=100)
axs[2].set_title("2-simplejo\n(Triángulo)")

# 3-simplejo (Proyección 2D)
puntos_tetraedro = np.array([[0, 0], [1, 0], [0.5, 0.866], [0.5, 0.3]])
aristas_tetraedro = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,3)]
for i, j in aristas_tetraedro:
    axs[3].plot([puntos_tetraedro[i,0], puntos_tetraedro[j,0]],
                [puntos_tetraedro[i,1], puntos_tetraedro[j,1]], 'b--')
axs[3].scatter(puntos_tetraedro[:,0], puntos_tetraedro[:,1], s=100)
axs[3].set_title("3-simplejo\n(Tetraedro\n sólido)")

for ax in axs:
    ax.set_xlim(-0.2, 1.2); ax.set_ylim(-0.2, 1.2); ax.axis('off')
plt.show()

### Caras
*   **Cara:** Si $\sigma$ es un simplejo definido por un conjunto de vértices, cualquier subconjunto no vacío de esos vértices define una **cara** de $\sigma$

### Complejos simpliciales
Un complejo simplicial es una colección de simplejos pegados de manera coherente.

> **Definición.**
> Un **complejo simplicial** $K$ es una colección de simplejos tal que:
> 1. Si $\sigma \in K$, entonces toda cara de $\sigma$ también pertenece a $K$.
> 2. Si $\sigma_1, \sigma_2 \in K$, entonces $\sigma_1 \cap \sigma_2$ es vacía o una cara común de ambas.

La condición 1 asegura que si tenemos un triángulo, también tenemos sus aristas y vértices. La condición 2 asegura que encajen correctamente (sin intersecciones parciales extrañas).

### Complejos simpliciales abstractos (C.S.A.)

En la computación, solemos usar una definición puramente combinatoria.

> **Definición.**
> Es un par $(V, K)$, donde $V$ es un conjunto de vértices y $K$ es una colección de subconjuntos no vacíos de $V$ (los simplejos) que satisface la propiedad de cerradura bajo contenciones (condición 1 anterior): si $\sigma \in K$ y $\tau \subseteq \sigma$, entonces $\tau \in K$.

**Ejemplo:**
Sea $V=\{1, 2, 3\}$.
*   $K_{hueco} = \{\{1\}, \{2\}, \{3\}, \{1, 2\}, \{2, 3\}, \{1, 3\}\}$. Representa el contorno de un triángulo (hueco).
*   $K_{relleno} = K_{hueco} \cup \{\{1, 2, 3\}\}$. Representa un triángulo relleno.

In [ ]:
# Visualización de C.S.A. (Hueco vs Relleno)

def visualizar_CSA(V, K, ax, titulo):
    # Posiciones fijas para los vértices 1, 2, 3
    pos = {1: (0, 0), 2: (1, 0), 3: (0.5, 0.866)}

    # Extraemos aristas (1-simplejos) y triángulos (2-simplejos)
    # Filtramos el conjunto vacío si está presente para la visualización
    aristas = [tuple(s) for s in K if len(s) == 2]
    triangulos = [tuple(s) for s in K if len(s) == 3]

    # Dibujamos los triángulos (relleno)
    for t in triangulos:
        # Verificamos que todos los vértices del triángulo tengan posición definida
        if all(v in pos for v in t):
            coords = [pos[v] for v in t]
            poly = plt.Polygon(coords, closed=True, facecolor='lightblue', alpha=0.5)
            ax.add_patch(poly)

    # Dibujamos el 1-esqueleto (Grafo)
    G = nx.Graph()
    G.add_nodes_from(V)
    G.add_edges_from(aristas)

    # Nos aseguramos de usar solo las posiciones de los nodos presentes en V que estén en 'pos'
    pos_actual = {k: v for k, v in pos.items() if k in V}

    # Si no tenemos posiciones para todos los nodos (caso general), usamos un layout por defecto
    if len(pos_actual) != len(V) or not pos_actual:
        pos_actual = nx.spring_layout(G, seed=42)

    nx.draw(G, pos_actual, ax=ax, with_labels=True, node_color='darkblue', edge_color='black', width=2)

    ax.set_title(titulo)
    # Ajuste de límites si usamos las posiciones fijas
    if 1 in V and 3 in V:
        ax.set_xlim(-0.1, 1.1); ax.set_ylim(-0.1, 1.0); ax.axis('equal')

V = {1, 2, 3}
# Usamos frozenset para los elementos de K ya que los sets no son hashables en Python.
# Incluimos el conjunto vacío (frozenset()) para coincidir con la definición dada en el texto.
K_hueco = {frozenset(), frozenset({1}), frozenset({2}), frozenset({3}), frozenset({1, 2}), frozenset({2, 3}), frozenset({1, 3})}
K_relleno = K_hueco.union({frozenset({1, 2, 3})})

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
visualizar_CSA(V, K_hueco, axs[0], "$K_{hueco}$")
visualizar_CSA(V, K_relleno, axs[1], "$K_{relleno}$")
plt.show()

## 📝 Ejercicio 1: Complejos Simpliciales Abstractos (Conceptual)

Considere el siguiente complejo simplicial abstracto:
$K = \{\{a\}, \{b\}, \{c\}, \{d\}, \{a, b\}, \{b, c\}, \{c, a\}, \{a, d\}\}$.

Responda las siguientes preguntas:

1.  ¿Cuál es la dimensión de $K$? (Recordatorio: Es la dimensión máxima de sus simplejos).
2.  ¿Contiene algún 2-simplejo (triángulo relleno)?
3.  Describa intuitivamente la forma de la realización geométrica de $K$.

### INICIO DE SU RESPUESTA ###

(Doble clic aquí para editar y escribir su respuesta)

**1. Dimensión de K:**

**2. ¿Contiene 2-simplejos?:**
(Justifique. ¿Está el conjunto {a, b, c} en K?)

**3. Descripción de la forma:**
(Pista: Identifique el ciclo formado por a, b, y c. ¿Qué hace la arista {a, d}?)

### FIN DE SU RESPUESTA ###

## 10. Filtraciones: De los datos al complejo

Si tenemos una nube de puntos (nuestros datos), ¿cómo construimos un complejo que capture su forma?

Si conectamos solo puntos muy cercanos, perdemos la estructura global. Si conectamos puntos muy lejanos, obtenemos un complejo demasiado denso.

La solución en TDA es analizar **todas las escalas** simultáneamente mediante una **filtración**.

> **Definición.**
> Una **filtración** es una sucesión anidada de complejos simpliciales:
> $$ K_0 \subseteq K_1 \subseteq K_2 \subseteq \dots \subseteq K_m $$

### Filtración de Vietoris–Rips (VR)

La construcción más utilizada en la práctica debido a su eficiencia computacional. Se basa en un parámetro de escala (distancia máxima) $\epsilon \geq 0$.

> **Definición: Complejo de Vietoris–Rips $\mathrm{VR}_\epsilon(X)$.**
>
> Dado un conjunto de puntos $X$ y una escala $\epsilon$:
> *   Un 1-simplejo (arista) $\{x_i, x_j\}$ existe si la distancia $d(x_i, x_j) \leq \epsilon$.
> *   Un k-simplejo $\{x_0, \dots, x_k\}$ existe si **todas** las distancias por pares entre sus vértices son $\leq \epsilon$.

Al aumentar $\epsilon$ desde 0 hasta infinito, obtenemos la **filtración de Vietoris-Rips**.

### Ejemplo visual de la filtración VR

Veamos cómo evoluciona el complejo VR al aumentar $\epsilon$ sobre una nube de puntos con una estructura interesante: un anillo ruidoso. Observaremos cómo se van conectando los puntos, cómo se forma el bucle central y cómo finalmente se rellena. (La ejecución puede tardar un minuto).

In [ ]:
# ---------------------------------------------------------------------
# 1) Datos: nube en forma de anillo (annulus) con leve ruido
# ---------------------------------------------------------------------
np.random.seed(42)
N_puntos_vr = 50
radio_vr     = 7.0
ruido_vr     = 0.6

ang = 2*np.pi*np.random.rand(N_puntos_vr)
rad = radio_vr + ruido_vr*np.random.randn(N_puntos_vr)
Puntos_Anillo_VR = np.c_[rad*np.cos(ang), rad*np.sin(ang)]

# Distancias euclidianas
D = squareform(pdist(Puntos_Anillo_VR))

# ---------------------------------------------------------------------
# 2) Utilidades para construir complejos VR
# ---------------------------------------------------------------------
def construir_aristas_y_triangulos(X, D, eps):
    """
    Para un epsilon dado, regresa:
      - segments: lista de segmentos [(x1,y1)-(x2,y2), ...] para 1-simplejos
      - tris: lista de triángulos como arreglos (3x2) para 2-simplejos
    """
    A = (D <= eps)  # matriz de adyacencia

    # Aristas (solo triángulo superior para evitar duplicados)
    ii, jj = np.where(np.triu(A, 1))
    segments = [np.vstack([X[i], X[j]]) for i, j in zip(ii, jj)]

    # Triángulos (i<j<k con todas las aristas presentes)
    tris = []
    for i, j, k in combinations(range(X.shape[0]), 3):
        if A[i, j] and A[i, k] and A[j, k]:
            tris.append(X[[i, j, k]])
    return segments, tris

def etapa_texto(eps):
    if eps < 1.5:
        return "Inicio — puntos aislados"
    elif eps < 3.0:
        return "Conexiones locales"
    elif eps < 5.0:
        return "Aparece un bucle (nace $H_1$)"
    elif eps < 8.0:
        return "Bucle persiste (persistencia de $H_1$)"
    else:
        return "Relleno completo (muere $H_1$)"

# ---------------------------------------------------------------------
# 3) Figura y artistas
# ---------------------------------------------------------------------
plt.rcParams['figure.dpi'] = 100
plt.rcParams['animation.html'] = 'jshtml'

fig, ax = plt.subplots(figsize=(5, 5), constrained_layout=True)
ax.set_aspect('equal', adjustable='box')
ax.set_xlim(-8, 8)
ax.set_ylim(-8, 8)
ax.set_axis_off()

# 0-simplejos (vértices)
sc = ax.scatter(Puntos_Anillo_VR[:, 0], Puntos_Anillo_VR[:, 1],
                s=36, marker='o', facecolor='tab:blue', edgecolor='k',
                linewidths=0.5, zorder=4)

# Bolas de radio eps/2
circulos = []
for p in Puntos_Anillo_VR:
    c = plt.Circle(p, radius=0.0, facecolor='red', alpha=0.12,
                   edgecolor='none', zorder=1)
    ax.add_patch(c)
    circulos.append(c)

# 1-simplejos (aristas) y 2-simplejos (triángulos)
edges_lc = LineCollection([], linewidths=1.2, alpha=0.85, zorder=3, capstyle='round')
ax.add_collection(edges_lc)
tris_pc = PolyCollection([], facecolors='lightgray', edgecolors='none', alpha=0.35, zorder=2)
ax.add_collection(tris_pc)

# Títulos
title_main = ax.text(0.5, 1.02, "Filtración de Vietoris–Rips en un anillo",
                     transform=ax.transAxes, ha='center', va='bottom',
                     fontsize=13, fontweight='bold')
title_sub  = ax.text(0.5, 0.98, "", transform=ax.transAxes, ha='center', va='top', fontsize=10)

# ---------------------------------------------------------------------
# 4) Cronometría y eps no uniforme
# ---------------------------------------------------------------------
DURACION_S = 14
FPS        = 15
interval_ms = int(1000 / FPS)

eps0, eps1, eps2, eps3, eps4, eps5 = 0.10, 1.5, 3.0, 5.0, 8.0, 12.0
n1, n2, n3, n4, n5 = 30, 50, 70, 40, 20  # total = 210 = DURACION_S * FPS

epsilon_values = np.r_[
    np.linspace(eps0, eps1, n1, endpoint=False),
    np.linspace(eps1, eps2, n2, endpoint=False),
    np.linspace(eps2, eps3, n3, endpoint=False),
    np.linspace(eps3, eps4, n4, endpoint=False),
    np.linspace(eps4, eps5, n5, endpoint=True),
]

# ---------------------------------------------------------------------
# 5) Funciones de animación
# ---------------------------------------------------------------------
def init():
    for c in circulos:
        c.set_radius(0.0)
    edges_lc.set_segments([])
    tris_pc.set_verts([])
    title_sub.set_text(f"ε = {epsilon_values[0]:.2f} — {etapa_texto(epsilon_values[0])}")
    return (*circulos, edges_lc, tris_pc, title_main, title_sub, sc)

def update(frame):
    eps = epsilon_values[frame]
    # Actualizar radios de las bolas (eps/2)
    r = eps / 2.0
    for c in circulos:
        c.set_radius(r)
    # 1- y 2-simplejos
    segments, tris = construir_aristas_y_triangulos(Puntos_Anillo_VR, D, eps)
    edges_lc.set_segments(segments)
    tris_pc.set_verts(tris)
    # Título
    title_sub.set_text(f"ε = {eps:.2f} — {etapa_texto(eps)}")
    return (*circulos, edges_lc, tris_pc, title_sub)

# ---------------------------------------------------------------------
# 6) Crear una sola animación
# ---------------------------------------------------------------------
ani = FuncAnimation(
    fig, update, init_func=init,
    frames=len(epsilon_values),
    interval=interval_ms,           # ~66.7 ms -> 15 fps
    blit=False, repeat=True, repeat_delay=800
)

plt.close()
HTML(ani.to_html5_video())

**Análisis de la filtración:**
Observe lo que sucede al aumentar $\epsilon$ (y por tanto, el radio de las bolas rojas, $r=\epsilon/2$):
1. Inicialmente, los puntos están aislados.
2. Se forman pequeñas conexiones y triángulos locales.
3. Las componentes se fusionan hasta formar una sola (Muerte de características en H0).
4. Se forma claramente la estructura del anillo (Nacimiento de una característica prominente en H1).
5. El complejo se vuelve más denso, pero el agujero central persiste.
6. Finalmente, $\epsilon$ es tan grande que se forman simplejos que cruzan el centro, rellenando el agujero (Muerte de la característica en H1).

### Complejo de Čech vs Vietoris-Rips

Existe otra construcción importante llamada el complejo de Čech. Está relacionado con el **Teorema del Nervio**, que conecta la geometría continua (cubiertas por abiertos) con la estructura combinatoria (el nervio).

> **Definición: Complejo de Čech $\check{C}_r(X)$.**
>
> Se define como el nervio de la cubierta de bolas de radio $r$ centradas en los puntos. Un k-simplejo existe si las $k+1$ bolas correspondientes tienen una **intersección común no vacía**.

**Diferencia clave:** En VR (con parámetro de distancia $\epsilon=2r$), solo requerimos que las bolas se intersequen **por pares**. En Čech (con parámetro $r$), requerimos que **todas** se intersequen en un punto común.

El complejo de Čech captura perfectamente la topología de la unión de las bolas (por el Teorema del Nervio), pero es computacionalmente más costoso. VR es una aproximación eficiente.

> **Teorema (Relación de Intercalado):** $C_r(X) \subseteq VR_r(X) \subseteq C_{2r}(X)$.

In [ ]:
# Visualización de la diferencia clave entre VR y Čech

# Consideremos 3 puntos formando un triángulo equilátero de lado 2.
Puntos_C_VR = np.array([[0, 0], [2, 0], [1, 1.732]])
radio = 1.05 # Radio ligeramente mayor a 1. (Epsilon = 2r = 2.1)

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

def configurar_grafica(ax, titulo):
    # Dibujamos las bolas (representando la cubierta)
    for p in Puntos_C_VR:
        circulo = plt.Circle(p, radio, color='gray', alpha=0.2, zorder=1)
        ax.add_patch(circulo)

    # Dibujamos los puntos (vértices del complejo)
    ax.scatter(Puntos_C_VR[:, 0], Puntos_C_VR[:, 1], s=50, zorder=4, c='blue')

    # Dibujamos las aristas (1-simplejos). Están presentes en ambos para este radio.
    for i in range(3):
        for j in range(i + 1, 3):
            ax.plot([Puntos_C_VR[i, 0], Puntos_C_VR[j, 0]],
                    [Puntos_C_VR[i, 1], Puntos_C_VR[j, 1]], 'k-', zorder=3, linewidth=2)

    ax.set_title(titulo)
    ax.axis('equal')
    # Ajustamos límites para mejor visualización
    ax.set_xlim(-1.5, 3.5); ax.set_ylim(-1.5, 3.0)
    ax.axis('off')


# Gráfica 1: Complejo de Čech
configurar_grafica(axs[0], "Complejo de Čech ($C_r$)")
# En Čech, el triángulo NO se rellena porque no hay intersección triple (el centro está vacío).
axs[0].text(1, 0.8, "Vacío\n(No hay intersección triple)", horizontalalignment='center', color='red', zorder=5, fontweight='bold')


# Gráfica 2: Complejo de Vietoris-Rips
configurar_grafica(axs[1], "Complejo de Vietoris-Rips ($VR_r$)")
# En VR, el triángulo SÍ se rellena porque todas las intersecciones por pares existen (distancia 2 <= 2r).
triangulo_VR = plt.Polygon(Puntos_C_VR, color='lightblue', alpha=0.6, zorder=2)
axs[1].add_patch(triangulo_VR)
axs[1].text(1, 0.8, "Relleno\n(Intersecciones por pares)", horizontalalignment='center', color='blue', zorder=5, fontweight='bold')

plt.suptitle(f"Comparación para radio r={radio} ($\epsilon$={2*radio})", fontsize=16)
plt.show()

## 11. Cadenas, operador frontera y homología

La homología es la herramienta de la topología algebraica para contar formalmente el número de agujeros de diferentes dimensiones. Para definirla, necesitamos maquinaria algebraica.

*(Nota: Trabajaremos con coeficientes en el campo finito $\mathbb{Z}_2$ (enteros módulo 2, es decir, 0 y 1). Esto simplifica los cálculos y evita tener que rastrear las **orientaciones de simplejos**, ya que en $\mathbb{Z}_2$ se cumple que $-1 = 1$, eliminando los problemas de signo.)*

### Grupos de cadenas (Chain groups)

> **Definición: Grupo de cadenas $C_k(K)$.**
>
> Es el espacio vectorial generado por los $k$-simplejos de $K$. Una **k-cadena** es una suma formal de $k$-simplejos con coeficientes en $\mathbb{Z}_2$.

**Interpretación con $\mathbb{Z}_2$:** Una k-cadena es simplemente un subconjunto de k-simplejos.

### El operador frontera (Boundary operator)

> **Definición: Operador frontera $\partial_k: C_k(K) \to C_{k-1}(K)$**
> Es una transformación lineal que mapea un $k$-simplejo a la suma (módulo 2) de sus caras de dimensión $(k-1)$.

**Ejemplos:**
*   $\partial_1(\{1, 2\}) = \{1\} + \{2\}$. (La frontera de una arista son sus vértices).
*   $\partial_2(\{1, 2, 3\}) = \{1, 2\} + \{2, 3\} + \{1, 3\}$. (La frontera de un triángulo son sus aristas).

### Propiedad fundamental

> **Teorema:** La composición de dos operadores frontera consecutivos es cero: $\partial_{k-1} \circ \partial_k = 0$. Es decir, $\partial^2=0$.

**Verificación intuitiva:** Calculemos la frontera de la frontera de un triángulo $\{1, 2, 3\}$.

$\partial_1(\partial_2(\{1, 2, 3\})) = \partial_1(\{1, 2\} + \{2, 3\} + \{1, 3\})$
$= (\{1\} + \{2\}) + (\{2\} + \{3\}) + (\{1\} + \{3\})$
$= 2\cdot\{1\} + 2\cdot\{2\} + 2\cdot\{3\}$

Como estamos en $\mathbb{Z}_2$, $2=0$. Por lo tanto, el resultado es $0$. La frontera no tiene frontera!

### Ciclos y Fronteras

*   **Ciclos ($Z_k$):** k-cadenas cuya frontera es cero. $Z_k(K) := \text{Ker}(\partial_k)$.
*   **Fronteras ($B_k$):** k-cadenas que son la frontera de alguna $(k+1)$-cadena. $B_k(K) := \text{Im}(\partial_{k+1})$.

Consecuencia de $\partial^2=0$: **Toda frontera es un ciclo.** ($B_k \subseteq Z_k$). Pero no todo ciclo es una frontera.

### Grupos de homología y Números de Betti

Queremos identificar los ciclos que **NO** son fronteras. Estos representan los verdaderos agujeros topológicos.

> **Definición: Grupo de homología $H_k(K)$.**
>
> Se define como el espacio vectorial cociente de los ciclos, módulo (cociente), las fronteras:
> $$ H_k(K) = \frac{Z_k(K)}{B_k(K)}.$$


> **Definición: Números de Betti $\beta_k(K)$.**
>
> Es la dimensión (rango) del k-ésimo grupo de homología: $\beta_k(K) = \dim(H_k(K))$.

**Interpretación intuitiva:**
*   $\beta_0$: Número de componentes conexas.
*   $\beta_1$: Número de bucles o lazos independientes (agujeros 1D).
*   $\beta_2$: Número de cavidades o vacíos independientes (agujeros 2D).

## 📝 Ejercicio 2: Números de Betti (Conceptual)

Determine los números de Betti $\beta_0, \beta_1, \beta_2$ para los siguientes objetos topológicos. Piense intuitivamente en el número de componentes, bucles y cavidades.

1.  Una esfera ($S^2$, la superficie de una pelota, hueca).
2.  Un toro ($T^2$, la superficie de una dona, hueca).
3.  Un círculo ($S^1$).
4.  Dos círculos disjuntos.

### INICIO DE SU RESPUESTA ###

(Doble clic aquí para editar y escribir su respuesta)

**1. Esfera ($S^2$):**
*   $\beta_0$ (Componentes):
*   $\beta_1$ (Bucles): (¿Cualquier bucle en la superficie se puede contraer a un punto?)
*   $\beta_2$ (Cavidades):

**2. Toro ($T^2$):**
*   $\beta_0$:
*   $\beta_1$: (Pista: Hay dos tipos de bucles fundamentales, uno alrededor del agujero central y otro a través del tubo).
*   $\beta_2$:

**3. Círculo ($S^1$):**
*   $\beta_0$:
*   $\beta_1$:
*   $\beta_2$: (Un círculo no encierra una cavidad 2D).

**4. Dos círculos disjuntos:**
*   $\beta_0$:
*   $\beta_1$:
*   $\beta_2$:

### FIN DE SU RESPUESTA ###

## 12. Topología de grafos (Puente al TDA)

Podemos conectar la teoría de grafos (Sesión 1) con la topología.

> **Definición: Complejo de cliques (Flag complex).**
>
> Dado un grafo $G=(V, E)$, el **complejo de cliques** $Cl(G)$ es el complejo simplicial donde un subconjunto de vértices $\sigma$ es un simplejo si y solo si $\sigma$ forma un **clique** en $G$ (esto es, todos los vértices están conectados entre sí).

**Ejemplo y distinción clave:**
*   Considere un **Grafo Triángulo** ($K_3$). Combinatoriamente tiene un ciclo. Pero en el complejo de cliques $Cl(K_3)$, este ciclo está relleno por el 2-simplejo correspondiente al clique de tamaño 3. Por lo tanto, topológicamente, $\beta_1(Cl(K_3))=0$.

**Esta distinción entre ciclos combinatorios (en el grafo) y agujeros topológicos (en el complejo de cliques) es crucial.**

*(Nota: La filtración de Vietoris-Rips es precisamente la construcción del complejo de cliques sobre el grafo definido por el umbral de distancia $\epsilon$.)*

In [ ]:
# Ejemplo: Complejo de Cliques de un grafo "Corbata de Moño"
# Grafo: Dos triángulos (1-2-3 y 3-4-5) unidos por el vértice 3.
G_bowtie = nx.Graph()
G_bowtie.add_edges_from([(1,2), (2,3), (1,3), (3,4), (4,5), (3,5)])

# Para construir el complejo de cliques, encontramos todos los cliques del grafo.
# NetworkX tiene la función nx.find_cliques() que encuentra los cliques maximales.
cliques_maximales = list(nx.find_cliques(G_bowtie))
print(f"Cliques maximales: {cliques_maximales}")

# El complejo de cliques Cl(G_bowtie) consiste en estos dos triángulos rellenos.

# Visualización
pos_bowtie = nx.spring_layout(G_bowtie, seed=42)
plt.figure(figsize=(6, 5))
ax = plt.gca()

# Rellenamos los cliques maximales (que son los triángulos)
for clique in cliques_maximales:
    coords = [pos_bowtie[v] for v in clique]
    poly = plt.Polygon(coords, closed=True, facecolor='lightblue', alpha=0.5)
    ax.add_patch(poly)

# Dibujamos el grafo (1-esqueleto)
nx.draw(G_bowtie, pos_bowtie, with_labels=True, node_color='darkblue', edge_color='black', width=2)
plt.title("Complejo de Cliques de la Corbata de Moño")
plt.show()

# Interpretación Topológica:
# Aunque el grafo tiene ciclos, todos están rellenos por los cliques.
# Por lo tanto, esperamos Beta_1 = 0.

## 📝 Ejercicio 3: Complejo de Cliques

Considere el grafo $G_{e3}$ que es un cuadrado simple: Vértices $\{1, 2, 3, 4\}$ y aristas $\{(1,2), (2,3), (3,4), (4,1)\}$.

**Instrucciones:**
1.  Cree el grafo $G_{e3}$ en NetworkX.
2.  Use `nx.find_cliques(G_e3)` para encontrar los cliques maximales.
3.  Interprete el resultado: ¿Cómo es la forma del complejo de cliques $Cl(G_{e3})$? ¿Cuáles son sus números de Betti esperados ($\beta_0, \beta_1$)?.

In [ ]:
### INICIO DE SU RESPUESTA (Código) ###

# 1. Cree el grafo G_e3 (Cuadrado simple).
G_e3 = nx.Graph()
E_e3 = [] # <- MODIFIQUE AQUÍ (Añada las 4 aristas: (1,2), ...)
G_e3.add_edges_from(E_e3)

# Verificamos que el grafo tenga la estructura esperada (4 nodos, 4 aristas)
if G_e3.number_of_nodes() == 4 and G_e3.number_of_edges() == 4:
    # Visualización del grafo
    nx.draw(G_e3, with_labels=True, node_color='lightblue')
    plt.title("G_e3: Cuadrado simple")
    plt.show()

    # 2. Encuentre los cliques maximales.
    # Use list(nx.find_cliques(G_e3))
    cliques_e3 = [] # <- MODIFIQUE AQUÍ

    if cliques_e3:
        print(f"Cliques maximales encontrados: {cliques_e3}")
else:
    print("Grafo G_e3 incompleto. Debe tener 4 vértices y 4 aristas formando un cuadrado.")

### FIN DE SU RESPUESTA (Código) ###

### INICIO DE SU RESPUESTA (Interpretación) ###

**3. Interpretación:**

(Doble clic para responder.)

*   **Forma de $Cl(G_{e3})$:** (Describa el complejo basándose en los cliques maximales. ¿Hay triángulos rellenos?)
*   **Números de Betti esperados:**
    *   $\beta_0$ =
    *   $\beta_1$ = (¿Tiene agujeros topológicos, es decir, ciclos que NO estén rellenos por cliques?)

### FIN DE SU RESPUESTA (Interpretación) ###

## 13. Homología persistente: La herramienta central de TDA

Ahora combinamos los conceptos de filtración y homología.

Tenemos una filtración (por ejemplo, VR al aumentar $\epsilon$):
$$ K_0 \subseteq K_1 \subseteq K_2 \subseteq \dots \subseteq K_m $$

> **Objetivo de la homología persistente (PH):**
> Rastrear cómo cambian los grupos de homología (los agujeros) a medida que avanzamos en la filtración.

### Nacimiento, Muerte y Persistencia

A medida que aumentamos $\epsilon$:

*   **Nacimiento ($b$):** Un nuevo agujero independiente se forma.
*   **Muerte ($d$):** Un agujero existente se rellena.
*   **Persistencia:** La duración $d-b$.

**Idea Clave:** Las características con **alta persistencia** son consideradas significativas (señal). Las características con **baja persistencia** son consideradas ruido topológico.

### Ejemplo: Cálculo de la Homología Persistente en un anillo

En los siguientes bloques de código:
1. Vamos a generar una nube de puntos en forma de anillo (un círculo con ruido).
2. Calcularemos su homología persistente usando la librería `ripser`.
3. Visualizaremos gráficamente sus características topológicas en un Diagrama de persistancia.
4. Se analizarán también las características topológicas en la representación del Barcode.


In [ ]:
# 1. Generar los datos (anillo)
N_puntos = 100
radio_anillo = 5
ruido = 0.5
np.random.seed(42)

# Generamos ángulos y radios con ruido
angulos = 2 * np.pi * np.random.rand(N_puntos)
radios = radio_anillo + ruido * np.random.randn(N_puntos)

# Convertimos a coordenadas cartesianas (x, y)
Datos_Anillo = np.vstack((radios * np.cos(angulos), radios * np.sin(angulos))).T

# Visualizamos los datos
plt.figure(figsize=(5, 5))
plt.scatter(Datos_Anillo[:, 0], Datos_Anillo[:, 1])
plt.title("Nube de puntos (anillo)"); plt.axis('equal'); plt.show()

In [ ]:
# 2. Calcular la Homología Persistente del anillo con ruido anterior (usando ripser)

# La función ripser() toma la nube de puntos y calcula la filtración VR y su Homología Persistente.
# 'maxdim=1' indica que queremos calcular H0 y H1 (componentes y bucles).

# Ignoramos una advertencia común de Ripser/Persim sobre la barra infinita de H0 al graficar.
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    resultado = ripser(Datos_Anillo, maxdim=1)

# El resultado contiene los 'diagramas', que son la información de nacimiento y muerte.
diagramas = resultado['dgms']

# diagramas[0] corresponde a H0. diagramas[1] corresponde a H1.
print(f"Número de características en H0: {len(diagramas[0])}")
print(f"Número de características en H1: {len(diagramas[1])}")

### Diagramas de persistencia (Persistence Diagrams)

> **Definición: Diagrama de persistencia.**
>
> Es una colección de puntos en el plano 2D, donde cada punto $(b, d)$ corresponde a una característica topológica con tiempo de nacimiento $b$ y tiempo de muerte $d$.

**Interpretación:**
*   El eje X es el nacimiento (Birth) y el eje Y la muerte (Death).
*   Todos los puntos están por encima de la diagonal $y=x$ (pues $d \geq b$).
*   **Puntos lejos de la diagonal = Alta persistencia = Significativos.**
*   **Puntos cerca de la diagonal = Baja persistencia = Ruido.**

In [ ]:
# 3. Visualizar el Diagrama de Persistencia del anillo con ruido

# Usamos la función plot_diagrams de la librería persim.
plt.figure(figsize=(6, 6))
plot_diagrams(diagramas, show=True)
# El gráfico muestra H0 en azul y H1 en naranja.

**Análisis del Diagrama:**

*   **H0 (Azul):** Muchos puntos nacen en 0 y mueren rápidamente. Hay un punto que persiste hasta el infinito (un punto azul muy alejado de la diagonal), representando la componente conexa global. $\implies \beta_0=1$.
*   **H1 (Naranja):** Varios puntos cerca de la diagonal (ruido) y **un punto naranja muy alejado de la diagonal**. Este punto representa el bucle principal del anillo. $\implies \beta_1=1$.

### Códigos de barras (Barcodes)

Una representación alternativa y equivalente es el código de barras. Es a menudo más intuitivo para visualizar cómo evolucionan las características a lo largo de la filtración (eje X). Las barras largas corresponden a alta persistencia.

In [ ]:
# 4. Códigos de barras (Barcodes) del anillo con ruido

def plot_barcodes_mpl(dgm, ax, *, xmax=None, lw=2.0, color="tab:blue",
                      titulo=None, ordenar_por_longitud=True):
    """
    Dibuja el código de barras para 'dgm' (array Nx2: [nacimiento, muerte]).
    - Ordena barras finitas por longitud (muerte - nacimiento) en orden descendente si 'ordenar_por_longitud' es True.
    - Las barras con muerte infinita se trazan punteadas hacia 'xmax' y con una flecha indicativa.
    """
    if dgm is None or dgm.size == 0:
        ax.set_title((titulo or "Código de barras") + " — sin datos")
        ax.set_yticks([])
        return

    finitas   = dgm[np.isfinite(dgm[:, 1])]
    infinitas = dgm[~np.isfinite(dgm[:, 1])]

    if xmax is None:
        xmax = 1.1 * float(finitas[:, 1].max()) if finitas.size else 1.0

    if finitas.size and ordenar_por_longitud:
        longitudes = finitas[:, 1] - finitas[:, 0]
        orden = np.argsort(longitudes, kind="stable")
        finitas = finitas[orden]

    y = 0
    for b, d in finitas:
        ax.hlines(y, b, d, color=color, linewidth=lw)
        y += 1

    for b, _ in infinitas:
        ax.hlines(y, b, xmax, color=color, linewidth=lw, linestyles="dashed")
        ax.annotate("", xy=(xmax, y), xytext=(max(b, xmax * 0.98), y),
                    arrowprops=dict(arrowstyle="->", lw=lw, color=color))
        y += 1

    ax.set_ylim(-1, y + 1)
    ax.set_yticks([])
    ax.set_xlim(0, xmax)
    if titulo:
        ax.set_title(titulo)

# Escala X común: máxima muerte finita entre H0 y H1
max_muerte_finita = 0.0
for dgm in diagramas:
    if dgm.size:
        muertes = dgm[np.isfinite(dgm[:, 1]), 1]
        if muertes.size:
            max_muerte_finita = max(max_muerte_finita, float(muertes.max()))
if max_muerte_finita == 0.0:
    max_muerte_finita = 1.0

# Figura (consistente con el diagrama de persistencia: H0 azul, H1 naranja)
fig, axs = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

# H0
if len(diagramas) >= 1 and diagramas[0].size:
    plot_barcodes_mpl(diagramas[0], axs[0],
                      xmax=1.1 * max_muerte_finita,
                      color="tab:blue",
                      titulo="Código de barras H0 (componentes conexas)",
                      ordenar_por_longitud=True)
else:
    axs[0].set_title("Código de barras H0 — sin datos")

# H1
if len(diagramas) >= 2 and diagramas[1].size:
    plot_barcodes_mpl(diagramas[1], axs[1],
                      xmax=1.1 * max_muerte_finita,
                      color="tab:orange",
                      titulo="Código de barras H1 (bucles)",
                      ordenar_por_longitud=True)
else:
    axs[1].set_title("Código de barras H1 — sin datos")

axs[1].set_xlabel(r"Parámetro de filtración ($\epsilon$)")  # raw string para evitar warnings por '\e'
plt.tight_layout()
plt.show()


**Análisis del Código de Barras:**

*   **H0:** Vemos muchas barras cortas que nacen en $\epsilon=0$ y mueren rápidamente. Esto representa cómo los puntos individuales se fusionan en componentes más grandes. La barra superior (que se extiende hasta el final del gráfico) representa la componente conexa final (la barra infinita).
*   **H1:** Vemos varias barras muy cortas (ruido) y una barra significativamente larga. Esta barra larga representa el bucle principal del anillo. Nace cuando $\epsilon$ es suficientemente grande para conectar el círculo, y muere cuando $\epsilon$ es tan grande que el centro se rellena.

## 📝 Ejercicio 4: Estabilidad frente a perturbaciones pequeñas

El Teorema de Estabilidad en TDA establece que pequeñas perturbaciones en los datos producen cambios acotados en los diagramas de persistencia (y, por ende, en los códigos de barras). En este ejercicio partiremos de la nube **`Datos_Anillo`** ya generada y construiremos una versión **perturbada por coordenada** para comparar.

**Instrucciones:**
1. **No regenere** la nube; use la variable existente **`Datos_Anillo`**.
2. Defina la amplitud del ruido por coordenada en **`amplitud_ruido`** (p. ej., `0.35`). El ruido será i.i.d. **Uniforme(-a, a)** y se suma **a cada coordenada** \((x,y)\).
3. Construya **`Datos_Anillo_Perturbado`** como `Datos_Anillo + ruido_xy`, donde `ruido_xy` tiene la **misma forma** que `Datos_Anillo` y entradas en \([-a,a]\). *(Opcional: fije una semilla, p. ej. `np.random.seed(43)`.)*
4. Visualice **lado a lado** (misma fila) las nubes **original** y **perturbada** con **idénticos límites de ejes** y **aspecto igualado**.
5. Calcule la **homología persistente (VR, `maxdim=1`)** para **ambas** nubes y obtenga sus diagramas: `dgms_orig` y `dgms_pert`.
6. Muestre **lado a lado** (segunda fila) los **diagramas de persistencia** con límites **[0, L]** comunes en ambos ejes.
7. Dibuje los **códigos de barras** con límite **x** común basado en la **máxima muerte finita global**:
   - **Fila 3:** **H0** — original (azul) vs. perturbado (azul).
   - **Fila 4:** **H1** — original (naranja) vs. perturbado (naranja).
8. **Varíe** `amplitud_ruido` (p. ej., 0.15, 0.35, 0.60) y **observe** qué barras cambian poco y cuáles se acercan a la diagonal.
9. **Explique** cómo estas observaciones ilustran el **Teorema de Estabilidad** y comente el efecto sobre $\beta_0$ (componentes) y $\beta_1$ (bucles).


In [ ]:
# 4. Estabilidad frente a perturbaciones pequeñas (usar Datos_Anillo y añadir ruido por coordenada)

# --------- 4.1 Parámetros de la perturbación por coordenada ----------
# Ruido uniforme i.i.d. por coordenada: U(-a, a)
amplitud_ruido = 0.35  # <- MODIFIQUE AQUÍ (pequeño para ver similitud y diferencia)
np.random.seed(43)     # <- MODIFIQUE AQUÍ si desea otra realización

# --------- 4.2 Comprobación y construcción de la nube perturbada ----------
assert 'Datos_Anillo' in globals(), "Datos_Anillo no está definido. Ejecute la celda 1 primero."
X_orig = Datos_Anillo
ruido_xy = np.random.uniform(-amplitud_ruido, amplitud_ruido, size=X_orig.shape)
X_pert = X_orig + ruido_xy

# --------- 4.3 Cálculo de PH (VR) para ambas nubes ----------
def ph_vr(X, maxdim=1):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        return ripser(X, maxdim=maxdim)['dgms']

# Usamos 'diagramas' si ya existe; si no, lo calculamos (robustez)
if 'diagramas' in globals() and isinstance(diagramas, (list, tuple)):
    dgms_orig = diagramas
else:
    dgms_orig = ph_vr(X_orig, maxdim=1)

dgms_pert = ph_vr(X_pert, maxdim=1)

# --------- 4.4 Función auxiliar: barcodes con Matplotlib ----------
def plot_barcodes_mpl(dgm, ax, *, xmax=None, lw=2.0, color="tab:blue",
                      titulo=None, ordenar_por_longitud=True):
    """
    Dibuja el código de barras para 'dgm' (array Nx2 con [nacimiento, muerte]).
    - Ordena las barras finitas por longitud (muerte - nacimiento) en orden DESCENDENTE si ordenar_por_longitud=True.
    - Las barras con muerte infinita se trazan punteadas hacia 'xmax' y con flecha.
    """
    if dgm is None or dgm.size == 0:
        ax.set_title((titulo or "Código de barras") + " — sin datos")
        ax.set_yticks([])
        return

    finitas   = dgm[np.isfinite(dgm[:, 1])]
    infinitas = dgm[~np.isfinite(dgm[:, 1])]

    if xmax is None:
        xmax = 1.1 * float(finitas[:, 1].max()) if finitas.size else 1.0

    if finitas.size and ordenar_por_longitud:
        longitudes = finitas[:, 1] - finitas[:, 0]
        orden = np.argsort(longitudes, kind="stable")
        finitas = finitas[orden]

    y = 0
    for b, d in finitas:
        ax.hlines(y, b, d, color=color, linewidth=lw)
        y += 1

    for b, _ in infinitas:
        ax.hlines(y, b, xmax, color=color, linewidth=lw, linestyles="dashed")
        ax.annotate("", xy=(xmax, y), xytext=(max(b, xmax * 0.98), y),
                    arrowprops=dict(arrowstyle="->", lw=lw, color=color))
        y += 1

    ax.set_ylim(-1, y + 1)
    ax.set_yticks([])
    ax.set_xlim(0, xmax)
    if titulo:
        ax.set_title(titulo)

# --------- 4.5 Límites comunes para comparabilidad ----------
# 4.5.1. Límites para los dispersogramas (misma ventana en ambas nubes)
m_disp = np.max(np.abs(np.vstack([X_orig, X_pert])))
xlim = (-1.10 * m_disp, 1.10 * m_disp)
ylim = xlim

# 4.5.2. Límites para diagramas de persistencia (mismo [0, L]×[0, L])
def limite_pd(*dgms_list):
    vals = []
    for dgms in dgms_list:
        for dgm in dgms:
            if dgm.size:
                vals.append(dgm[np.isfinite(dgm)].max())
    return 1.05 * (max(vals) if vals else 1.0)

L_pd = limite_pd(dgms_orig, dgms_pert)

# 4.5.3. Límite X común para barcodes (máxima muerte finita global)
def max_muerte_finita_global(*dgms_list):
    M = 0.0
    for dgms in dgms_list:
        for dgm in dgms:
            if dgm.size:
                fin = dgm[np.isfinite(dgm[:, 1]), 1]
                if fin.size:
                    M = max(M, float(fin.max()))
    return 1.1 * (M if M > 0 else 1.0)

xmax_bar = max_muerte_finita_global(dgms_orig, dgms_pert)

# --------- 4.6 Figura 4×2: (1) dispersogramas, (2) PDs, (3) Barcodes H0, (4) Barcodes H1 ----------
fig = plt.figure(figsize=(12, 16))
gs  = gridspec.GridSpec(4, 2, height_ratios=[1.6, 1.3, 1.3, 1.3], figure=fig)

# (Fila 1) Nubes original vs perturbada
ax_scat_o = fig.add_subplot(gs[0, 0])
ax_scat_o.scatter(X_orig[:, 0], X_orig[:, 1],
                  s=22, marker='o', facecolor='tab:blue', edgecolor='k', linewidths=0.4)
ax_scat_o.set_title("Nube original (Datos_Anillo)")
ax_scat_o.set_aspect('equal', adjustable='box')
ax_scat_o.set_xlim(*xlim); ax_scat_o.set_ylim(*ylim)

ax_scat_p = fig.add_subplot(gs[0, 1])
ax_scat_p.scatter(X_pert[:, 0], X_pert[:, 1],
                  s=22, marker='o', facecolor='tab:blue', edgecolor='k', linewidths=0.4)
ax_scat_p.set_title(f"Nube perturbada (ruido U[-{amplitud_ruido:.2f},{amplitud_ruido:.2f}])")
ax_scat_p.set_aspect('equal', adjustable='box')
ax_scat_p.set_xlim(*xlim); ax_scat_p.set_ylim(*ylim)

# (Fila 2) Diagramas de persistencia
ax_pd_o = fig.add_subplot(gs[1, 0])
plot_diagrams(dgms_orig, ax=ax_pd_o, show=False, lifetime=False, legend=False)
ax_pd_o.set_title("Diagrama de persistencia — original")
ax_pd_o.set_xlim(0, L_pd); ax_pd_o.set_ylim(0, L_pd)

ax_pd_p = fig.add_subplot(gs[1, 1])
plot_diagrams(dgms_pert, ax=ax_pd_p, show=False, lifetime=False, legend=False)
ax_pd_p.set_title("Diagrama de persistencia — perturbado")
ax_pd_p.set_xlim(0, L_pd); ax_pd_p.set_ylim(0, L_pd)

# (Fila 3) Barcodes H0
ax_b0_o = fig.add_subplot(gs[2, 0])
if len(dgms_orig) > 0 and dgms_orig[0].size:
    plot_barcodes_mpl(dgms_orig[0], ax_b0_o, xmax=xmax_bar,
                      color="tab:blue", titulo="Código de barras H0 — original")
else:
    ax_b0_o.set_title("Código de barras H0 — original (sin datos)")

ax_b0_p = fig.add_subplot(gs[2, 1])
if len(dgms_pert) > 0 and dgms_pert[0].size:
    plot_barcodes_mpl(dgms_pert[0], ax_b0_p, xmax=xmax_bar,
                      color="tab:blue", titulo="Código de barras H0 — perturbado")
else:
    ax_b0_p.set_title("Código de barras H0 — perturbado (sin datos)")

# (Fila 4) Barcodes H1
ax_b1_o = fig.add_subplot(gs[3, 0])
if len(dgms_orig) > 1 and dgms_orig[1].size:
    plot_barcodes_mpl(dgms_orig[1], ax_b1_o, xmax=xmax_bar,
                      color="tab:orange", titulo="Código de barras H1 — original")
else:
    ax_b1_o.set_title("Código de barras H1 — original (sin datos)")

ax_b1_p = fig.add_subplot(gs[3, 1])
if len(dgms_pert) > 1 and dgms_pert[1].size:
    plot_barcodes_mpl(dgms_pert[1], ax_b1_p, xmax=xmax_bar,
                      color="tab:orange", titulo="Código de barras H1 — perturbado")
else:
    ax_b1_p.set_title("Código de barras H1 — perturbado (sin datos)")

# Etiqueta común para los barcodes de la última fila
ax_b1_p.set_xlabel(r"Parámetro de filtración ($\epsilon$)")  # raw string para evitar warnings por '\e'
plt.tight_layout()
plt.show()


### INICIO DE SU RESPUESTA (Conceptual) ###

**Pregunta:** ¿Cómo cambiaron el diagrama de persistencia y el código de barras al aumentar el ruido? Específicamente, ¿qué pasó con la característica de H1 (punto en el diagrama, barra en el barcode) que representa el bucle principal? ¿Sigue siendo identificable?

(Doble clic para responder. Comente si la barra principal de H1 se acortó (menor persistencia) y si aparecieron más barras de ruido.)

### FIN DE SU RESPUESTA (Conceptual) ###

## 📝 Ejercicio 5: Análisis de una estructura compleja (Figura 8)

Ahora analizaremos una nube de puntos con forma de "Figura 8" (lemniscata). Esperamos detectar una componente conexa y dos bucles prominentes.

**Instrucciones:**
1.  Los datos (`Datos_Fig8`) ya están generados en el bloque de código. Visualícelos.
2.  Calcule la homología persistente (H0 y H1) usando `ripser()`.
3.  Visualice el diagrama de persistencia usando `plot_diagrams()`.
4.  Interprete el resultado y responda la pregunta.

In [ ]:
# 1. Generación de datos (Figura 8)
N_fig8 = 150
np.random.seed(42)
t = np.random.rand(N_fig8) * 2 * np.pi
# Ecuación paramétrica para la lemniscata de Bernoulli (simplificada)
Datos_Fig8 = np.array([np.cos(t), np.sin(2*t)/2]).T
# Añadimos un poco de ruido
Datos_Fig8 += np.random.normal(0, 0.05, Datos_Fig8.shape)

# Visualización
plt.figure(figsize=(6, 4))
plt.scatter(Datos_Fig8[:, 0], Datos_Fig8[:, 1])
plt.title("Nube de puntos (Figura 8)")
plt.axis('equal')
plt.show()

### INICIO DE SU RESPUESTA (Código) ###

# 2. Calcular la Homología Persistente (maxdim=1)
resultado_fig8 = None # <- MODIFIQUE AQUÍ (Use ripser() en Datos_Fig8)

# Si desea usar el bloque 'with' para manejar las advertencias (recomendado):
# with warnings.catch_warnings():
#    warnings.simplefilter("ignore", category=UserWarning)
#    resultado_fig8 = ripser(Datos_Fig8, maxdim=1)


# 3. Visualizar el Diagrama de Persistencia
if resultado_fig8:
    diagramas_fig8 = resultado_fig8['dgms']
    plt.figure(figsize=(6, 6))
    # <- MODIFIQUE AQUÍ (Use plot_diagrams() en diagramas_fig8)
    # Ejemplo: plot_diagrams(diagramas_fig8)

    plt.title("Diagrama de Persistencia (Figura 8)")
else:
    print("Por favor, complete el paso 2.")

### FIN DE SU RESPUESTA (Código) ###

### INICIO DE SU RESPUESTA (Interpretación) ###

**4. Interpretación del diagrama:**

(Doble clic para responder.)

**Análisis H0:** ¿Cuántas componentes conexas significativas detecta?

**Análisis H1:** ¿Cuántos bucles significativos detecta? (Mire los puntos naranjas lejos de la diagonal).

**Conclusión:** ¿Coincide el diagrama con la estructura esperada de la Figura 8?

### FIN DE SU RESPUESTA (Interpretación) ###

## Cierre del módulo

Hemos recorrido los fundamentos de la Teoría de Grafos y el Análisis Topológico de Datos. Aprendimos cómo los grafos modelan relaciones y cómo el TDA, utilizando herramientas como complejos simpliciales y homología persistente, nos permite descubrir la forma y estructura subyacente de los datos de manera robusta al ruido y a múltiples escalas.

Estas herramientas forman la base teórica para aplicaciones avanzadas en modelado matemático y simulación en diversas disciplinas.